In [1]:
import requests
import json
from datetime import datetime
import teradatasql
import pandas as pd


In [2]:
### Connect to Teradata and get all deals that need to be checked in Deal Catalogue
connect = teradatasql.connect(host='tdwd.group.on',user='ub_intl_sales_ops',password='BA_intl_grp_3')
input_data = pd.read_sql("SELECT deal_uuid,contract_number FROM sandbox.EMEA_Travel_deals_dim WHERE deal_ends_at >= Current_Date AND booking_voucher = 'Booking' AND country_name IN ('BE','DE','ES','FR','GB', 'IE', 'IT', 'NL', 'PL','AU') GROUP BY 1,2",connect)

live_deals = input_data.values.tolist()


c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
### Set up additional variables
now = datetime.utcnow().strftime('%Y-%m-%d')

errors = []
divisions = []


In [4]:
### For all deal_uuids, check the deal_catalog values and add them to the Divisions variable
for i in range(1,len(live_deals)):
    deal_uuid = live_deals[i][0]
    contract_id = live_deals[i][1]

    url = f'http://deal-catalog.snc1/deal_catalog/v2/deals/{deal_uuid}?clientId=f183e5fbee1bb4cd-display-ads'
    try:
        r = requests.get(url)
        json_deal_catalogue = json.loads(r.text)
        DRC = []
        DRC = json_deal_catalogue["deal"]["distributionRegionCodes"]
        for c in DRC:
            country = c
            divisions.append([now,deal_uuid,contract_id,country])
    except:
        errors.append(deal_uuid)
        pass
    

In [5]:
df = pd.DataFrame(divisions)
df.columns = ['load_date','deal_uuid','contract_id','country_code']
    


In [6]:
### Upload data into Teradata
for row in divisions:
    load_date = row[0]
    deal_uuid = row[1]
    contract_id = row[2]
    country_code = row[3]

    query = f"INSERT INTO sandbox.getaways_xnt_drc ('{load_date}','{deal_uuid}','{contract_id}','{country_code}')"
    
    pd.read_sql_query(query,connect)
       
print("Success")


c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas

Success
